# Part-of-Speech разметка, NER, извлечение отношений

## Задание 1. Написать теггер на данных с русским языком

1. проверить UnigramTagger, BigramTagger, TrigramTagger и их комбмнации
2. написать свой теггер как на занятии, попробовать разные векторайзеры, добавить знание не только букв но и слов
3. сравнить все реализованные методы сделать выводы

In [1]:
!pip install pyconll

In [2]:
import pyconll

In [3]:
!mkdir datasets
!wget -O ./datasets/ru_syntagrus-ud-train.conllu https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-train.conllu
!wget -O ./datasets/ru_syntagrus-ud-dev.conllu https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-dev.conllu

--2021-07-25 04:49:52--  https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-train.conllu
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 81039282 (77M) [text/plain]
Saving to: ‘./datasets/ru_syntagrus-ud-train.conllu’

./datasets/ru_synta 100%[===================>]  77.28M   176MB/s    in 0.4s    

2021-07-25 04:49:53 (176 MB/s) - ‘./datasets/ru_syntagrus-ud-train.conllu’ saved [81039282/81039282]

--2021-07-25 04:49:53--  https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-dev.conllu
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercont

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import CountVectorizer, HashingVectorizer, TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

from gensim.models import Word2Vec, FastText

from nltk.tag import DefaultTagger
from nltk.tag import UnigramTagger
from nltk.tag import BigramTagger, TrigramTagger
from nltk.tag import RegexpTagger
from nltk.corpus import names
import nltk
nltk.download('names')

[nltk_data] Downloading package names to /root/nltk_data...
[nltk_data]   Unzipping corpora/names.zip.


True

In [5]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder

from scipy.sparse import hstack

In [6]:
import warnings
warnings.filterwarnings("ignore")

In [7]:
full_train = pyconll.load_from_file('datasets/ru_syntagrus-ud-train.conllu')
full_test = pyconll.load_from_file('datasets/ru_syntagrus-ud-dev.conllu')

In [8]:
for sent in full_train[:2]:
    for token in sent:
        print(token.form, token.upos)
    print()

Анкета NOUN
. PUNCT

Начальник NOUN
областного ADJ
управления NOUN
связи NOUN
Семен PROPN
Еремеевич PROPN
был AUX
человек NOUN
простой ADJ
, PUNCT
приходил VERB
на ADP
работу NOUN
всегда ADV
вовремя ADV
, PUNCT
здоровался VERB
с ADP
секретаршей NOUN
за ADP
руку NOUN
и CCONJ
иногда ADV
даже PART
писал VERB
в ADP
стенгазету NOUN
заметки NOUN
под ADP
псевдонимом NOUN
" PUNCT
Муха NOUN
" PUNCT
. PUNCT



### 1.1 проверить UnigramTagger, BigramTagger, TrigramTagger и их комбмнации

In [9]:
fdata_train = []
for sent in full_train[:]:
    fdata_train.append([(token.form, token.upos) for token in sent])
    
fdata_test = []
for sent in full_test[:]:
    fdata_test.append([(token.form, token.upos) for token in sent])
    
fdata_sent_test = []
for sent in full_test[:]:
    fdata_sent_test.append([token.form for token in sent])
    
    
MAX_SENT_LEN = max(len(sent) for sent in full_train)
MAX_ORIG_TOKEN_LEN = max(len(token.form) for sent in full_train for token in sent)
print('Наибольшая длина предложения', MAX_SENT_LEN)
print('Наибольшая длина токена', MAX_ORIG_TOKEN_LEN)

Наибольшая длина предложения 205
Наибольшая длина токена 47


In [10]:
default_tagger = nltk.DefaultTagger('NOUN')
default_tagger.evaluate(fdata_test)

0.23568564014423887

In [11]:
unigram_tagger = UnigramTagger(fdata_train)
unigram_tagger.evaluate(fdata_test)

0.8772537323492737

In [12]:
bigram_tagger = BigramTagger(fdata_train, backoff=unigram_tagger)
bigram_tagger.evaluate(fdata_test)

0.8829828463586425

In [13]:
trigram_tagger = TrigramTagger(fdata_train, backoff=bigram_tagger)
trigram_tagger.evaluate(fdata_test)

0.882081353418933

In [14]:
def backoff_tagger(train_sents, tagger_classes, backoff=None):
    for cls in tagger_classes:
        backoff = cls(train_sents, backoff=backoff)
    return backoff


backoff = DefaultTagger('NOUN') 
tag = backoff_tagger(fdata_train,  
                     [
                      UnigramTagger, 
                      BigramTagger, 
                      TrigramTagger
                     ],  
                     backoff = backoff) 
  
tag.evaluate(fdata_test)

0.9119991237825633

## 1.2 написать свой теггер как на занятии, попробовать разные векторайзеры, добавить знание не только букв но и слов

In [15]:
train_tok = []
train_label = []
for sent in fdata_train[:]:
    for tok in sent:
        train_tok.append(tok[0])
        train_label.append('NO_TAG' if tok[1] is None else tok[1])
        
test_tok = []
test_label = []
for sent in fdata_test[:]:
    for tok in sent:
        test_tok.append(tok[0])
        test_label.append('NO_TAG' if tok[1] is None else tok[1])
        
        
le = LabelEncoder()
train_enc_labels = le.fit_transform(train_label) 
test_enc_labels = le.transform(test_label)
le.classes_

array(['ADJ', 'ADP', 'ADV', 'AUX', 'CCONJ', 'DET', 'INTJ', 'NOUN',
       'NO_TAG', 'NUM', 'PART', 'PRON', 'PROPN', 'PUNCT', 'SCONJ', 'SYM',
       'VERB', 'X'], dtype='<U6')

In [16]:
for vectorizer in [CountVectorizer, HashingVectorizer, TfidfVectorizer]:

    scaler = StandardScaler(with_mean=False)
    coder = vectorizer(ngram_range=(1, 5), analyzer='char')
    

    X_train = coder.fit_transform(train_tok)
    X_test = coder.transform(test_tok)
    
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.fit_transform(X_test)    
    
    
    print(X_train.shape)
    lr = LogisticRegression(random_state=0, max_iter = 100, n_jobs=7)
    lr.fit(X_train, train_enc_labels)

    pred = lr.predict(X_test)

    print(vectorizer, accuracy_score(test_enc_labels, pred))

(871526, 149809)
<class 'sklearn.feature_extraction.text.CountVectorizer'> 0.943644053516665
(871526, 1048576)
<class 'sklearn.feature_extraction.text.HashingVectorizer'> 0.9471826239342163
(871526, 149809)
<class 'sklearn.feature_extraction.text.TfidfVectorizer'> 0.9487749806221144


In [17]:
for vectorizer in [CountVectorizer, HashingVectorizer, TfidfVectorizer]:

    scaler = StandardScaler(with_mean=False)
    coder = vectorizer(ngram_range=(1, 5), analyzer='word')
    

    X_train = coder.fit_transform(train_tok)
    X_test = coder.transform(test_tok)
    
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.fit_transform(X_test)    
    
    
    print(X_train.shape)
    lr = LogisticRegression(random_state=0, max_iter = 100, n_jobs=7)
    lr.fit(X_train, train_enc_labels)

    pred = lr.predict(X_test)

    print(vectorizer, accuracy_score(test_enc_labels, pred))

(871526, 99485)
<class 'sklearn.feature_extraction.text.CountVectorizer'> 0.7531847133757962
(871526, 1048576)
<class 'sklearn.feature_extraction.text.HashingVectorizer'> 0.7722255922892866
(871526, 99485)
<class 'sklearn.feature_extraction.text.TfidfVectorizer'> 0.7532605398847437


In [18]:
scaler = StandardScaler(with_mean=False)
coder_1 = TfidfVectorizer(ngram_range=(1, 5), analyzer='char')
coder_2 = HashingVectorizer(ngram_range=(1, 5), analyzer='word')

X_train_1 = coder_1.fit_transform(train_tok)
X_test_1 = coder_1.transform(test_tok)

X_train_2 = coder_2.fit_transform(train_tok)
X_test_2 = coder_2.transform(test_tok)


X_train = hstack((X_train_1,X_train_2))
X_test = hstack((X_test_1,X_test_2))

X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)    


print(X_train.shape)
lr = LogisticRegression(random_state=0, max_iter = 100, n_jobs=7)
lr.fit(X_train, train_enc_labels)

pred = lr.predict(X_test)

print('TfidfVectorizer_char + HashingVectorizer_word :', accuracy_score(test_enc_labels, pred))

(871526, 1198385)
TfidfVectorizer_char + HashingVectorizer_word : 0.9441327132409935


__Выводы:__

Для nltk.tag лучший вариант это: Комбинация из DefaultTagger UnigramTagger BigramTagger TrigramTagger
0.9119991237825633

Для Vectorizer лучший вариант это: Комбинация из LogisticRegression поверх TfidfVectorizer при условии analyzer='char'
0.9487749806221144

## Задание 2. Проверить насколько хорошо работает NER
данные брать из http://www.labinform.ru/pub/named_entities/
1. взять нер из nltk
2. проверить deeppavlov
3. написать свой нер попробовать разные подходы:
 - передаём в сетку токен и его соседей
 - передаём в сетку только токен
4. сделать выводы по вашим экспериментам какой из подходов успешнее справляется


## 2.1 взять нер из nltk

In [19]:
!pip install corus

     |████████████████████████████████| 83 kB 1.4 MB/s 


In [20]:
import corus

In [21]:
from corus import load_ne5

In [22]:
!wget http://www.labinform.ru/pub/named_entities/collection5.zip

--2021-07-25 05:29:43--  http://www.labinform.ru/pub/named_entities/collection5.zip
Resolving www.labinform.ru (www.labinform.ru)... 80.240.100.4
Connecting to www.labinform.ru (www.labinform.ru)|80.240.100.4|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1899530 (1.8M) [application/zip]
Saving to: ‘collection5.zip’

collection5.zip     100%[===================>]   1.81M   443KB/s    in 4.2s    

2021-07-25 05:29:47 (443 KB/s) - ‘collection5.zip’ saved [1899530/1899530]



In [23]:
import nltk
nltk.download('maxent_ne_chunker')
nltk.download('words')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping chunkers/maxent_ne_chunker.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [24]:
!ls

collection5.zip  datasets  sample_data


In [ ]:
# !unzip collection5.zip

In [26]:
path = 'Collection5/'
records = load_ne5(path)

In [27]:
document = next(records).text

In [28]:
document

'Главный борец с мафией в СНГ задержан за хищение 46 миллионов долларов\r\n\r\nПо подозрению в хищении 46 миллионов долларов в Москве задержан директор Бюро по координации борьбы с оргпреступностью и другими опасными преступлениями в государствах СНГ (БКБОП) генерал-лейтенант милиции Александр Боков. По словам представителя СК РФ Владимира Маркина, уголовное дело возбуждено по статье "мошенничество", сообщает РИА Новости.\r\n\r\nДо работы в исполнительном комитете СНГ, Боков руководил организационно-инспекторским управлением МВД РФ. По словам Маркина, генерал был задержан в ночь на 19 января.\r\n\r\nРанее источник в правоохранительных органах рассказал агентству "Интерфакс", что Бокова задержали за вымогательство и хищение 10 миллионов долларов у руководителя одной из крупных транспортных компаний.\r\n\r\nЕще один источник заявил РАПСИ, что о преступной деятельности милицейского генерала стало известно при разработке оперативных материалов. Вместе с Боковым были задержаны трое его сооб

In [29]:
{(' '.join(c[0] for c in chunk), chunk.label() ) for chunk in nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(document))) if hasattr(chunk, 'label') }

{('БКБОП', 'ORGANIZATION'),
 ('Боков', 'PERSON'),
 ('Бокова', 'PERSON'),
 ('Боковым', 'PERSON'),
 ('Главный', 'PERSON'),
 ('Источники', 'PERSON'),
 ('Кроме', 'PERSON'),
 ('Михаил Креймер', 'PERSON'),
 ('Москве', 'PERSON'),
 ('СНГ', 'ORGANIZATION'),
 ('Сергей Степанов', 'PERSON')}

## 2.2. проверить deeppavlov

In [ ]:
# !pip install deeppavlov

In [ ]:
# не пошла установка

## 2.3 написать свой нер попробовать разные подходы:
- передаём в сетку токен и его соседей
- передаём в сетку только токен

In [30]:
!pip install razdel

In [31]:
from razdel import tokenize

In [32]:
records = corus.load_ne5(path)
words_docs = []
for ix, rec in enumerate(records):
    words = []
    for token in tokenize(rec.text):
       
        result = 'None'        
        
        for item in rec.spans:            
            if (token.start >= item.start) and (token.stop <= item.stop) and (item.type == 'PER'):
                result = 'PER'
                break
            if (token.start >= item.start) and (token.stop <= item.stop) and (item.type == 'ORG'):
                result = 'ORG'
                break
            if (token.start >= item.start) and (token.stop <= item.stop) and (item.type == 'MEDIA'):
                result = 'MEDIA'
                break
            if (token.start >= item.start) and (token.stop <= item.stop) and (item.type == 'LOC'):
                result = 'LOC'
                break
            if (token.start >= item.start) and (token.stop <= item.stop) and (item.type == 'GEOPOLIT'):
                result = 'GEOPOLIT'
                break
                
    
        words.append([token.text, result])
    words_docs.extend(words)

In [33]:
df_words = pd.DataFrame(words_docs, columns=['word', 'tag'])

In [34]:
df_words['tag'].value_counts()

None        219214
PER          21200
ORG          13651
LOC           4568
GEOPOLIT      4356
MEDIA         2482
Name: tag, dtype: int64

In [35]:
df_words.head(5)

,word,tag
0,Главный,None
1,борец,None
2,с,None
3,мафией,None
4,в,None


In [36]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D, GlobalMaxPooling1D, Conv1D, GRU, LSTM, Dropout, Input
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

In [37]:
from sklearn import model_selection, preprocessing, linear_model

train_x, valid_x, train_y, valid_y = model_selection.train_test_split(df_words['word'], df_words['tag'])

# labelEncode целевую переменную
encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(train_y)
valid_y = encoder.fit_transform(valid_y)

In [38]:
train_data = tf.data.Dataset.from_tensor_slices((train_x, train_y))
valid_data = tf.data.Dataset.from_tensor_slices((valid_x, valid_y))

train_data = train_data.batch(16)
valid_data = valid_data.batch(16)

In [39]:
AUTOTUNE = tf.data.AUTOTUNE

train_data = train_data.cache().prefetch(buffer_size=AUTOTUNE)
valid_data = valid_data.cache().prefetch(buffer_size=AUTOTUNE)

In [40]:
def custom_standardization(input_data):
        return input_data

def data_prep(train_data, seq_len=1, vocab_size = 30000):    
    
    vocab_size = 30000
    #seq_len = 1

    vectorize_layer = TextVectorization(
        standardize=custom_standardization,
        max_tokens=vocab_size,
        output_mode='int',
        output_sequence_length=seq_len)


    # Make a text-only dataset (no labels) and call adapt to build the vocabulary.
    text_data = train_data.map(lambda x, y: x)
    vectorize_layer.adapt(text_data)
    return vectorize_layer

In [41]:
embedding_dim = 64

class modelNER(tf.keras.Model):
    def __init__(self):
        super(modelNER, self).__init__()
        self.emb = Embedding(vocab_size, embedding_dim)
        self.gPool = GlobalMaxPooling1D()
        self.fc1 = Dense(300, activation='relu')
        self.fc2 = Dense(50, activation='relu')
        self.fc3 = Dense(len(df_words['tag'].value_counts()), activation='softmax')

    def call(self, x):
        x = vectorize_layer(x)
        x = self.emb(x)
        pool_x = self.gPool(x)
        
        fc_x = self.fc1(pool_x)
        fc_x = self.fc2(fc_x)
        
        concat_x = tf.concat([pool_x, fc_x], axis=1)
        return self.fc3(concat_x)

In [42]:
vocab_size = 30000
vectorize_layer = data_prep(train_data, seq_len = 1, vocab_size = vocab_size)


mmodel = modelNER()
mmodel.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])
mmodel.fit(train_data, validation_data=valid_data, epochs=5)

Epoch 1/5
12444/12444 [==============================] - 272s 22ms/step - loss: 0.2840 - accuracy: 0.9172 - val_loss: 0.2037 - val_accuracy: 0.9402
Epoch 2/5
12444/12444 [==============================] - 315s 25ms/step - loss: 0.1223 - accuracy: 0.9633 - val_loss: 0.2131 - val_accuracy: 0.9421
Epoch 3/5
12444/12444 [==============================] - 309s 25ms/step - loss: 0.1077 - accuracy: 0.9657 - val_loss: 0.2350 - val_accuracy: 0.9416
Epoch 4/5
12444/12444 [==============================] - 308s 25ms/step - loss: 0.1029 - accuracy: 0.9665 - val_loss: 0.2633 - val_accuracy: 0.9413
Epoch 5/5
12444/12444 [==============================] - 306s 25ms/step - loss: 0.1004 - accuracy: 0.9670 - val_loss: 0.2722 - val_accuracy: 0.9345


In [43]:
vocab_size = 30000
vectorize_layer = data_prep(train_data, seq_len = 3, vocab_size = vocab_size)


mmodel = modelNER()
mmodel.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])
mmodel.fit(train_data, validation_data=valid_data, epochs=5)

Epoch 1/5
12444/12444 [==============================] - 307s 25ms/step - loss: 0.2939 - accuracy: 0.9142 - val_loss: 0.2122 - val_accuracy: 0.9394
Epoch 2/5
12444/12444 [==============================] - 295s 24ms/step - loss: 0.1240 - accuracy: 0.9631 - val_loss: 0.2587 - val_accuracy: 0.8865
Epoch 3/5
12444/12444 [==============================] - 296s 24ms/step - loss: 0.1090 - accuracy: 0.9656 - val_loss: 0.2676 - val_accuracy: 0.8870
Epoch 4/5
12444/12444 [==============================] - 292s 23ms/step - loss: 0.1040 - accuracy: 0.9664 - val_loss: 0.2411 - val_accuracy: 0.9410
Epoch 5/5
12444/12444 [==============================] - 278s 22ms/step - loss: 0.1011 - accuracy: 0.9668 - val_loss: 0.2390 - val_accuracy: 0.9411


#### Вывод.

Длина последовательности практически не влияет на результат